In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import MaxPool2D, GlobalMaxPool2D, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, Concatenate,Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2, l1

In [4]:
import pandas as pd
import numpy as np
import os
import cv2
from joblib import load, dump
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,KFold,RepeatedKFold,RepeatedStratifiedKFold, train_test_split
from aggmap import AggMap, AggMapNet

In [5]:
source_path = '/raid/lzyt_dir/Data_DRIAMS/Redo'
Xpath = '/mnt/lzyt/DRIAMS/A'
specie = 'Ecoli'
gpuid = '0'

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"]=gpuid
os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"
physical_gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_gpus[0], True)    #动态调用GPU

In [2]:
def Inception(inputs, units = 8, strides = 1):
    """
    naive google inception block
    """
    x1 = Conv2D(units, 5, padding='same', activation = 'relu', strides = strides)(inputs)
    x2 = Conv2D(units, 3, padding='same', activation = 'relu', strides = strides)(inputs)
    x3 = Conv2D(units, 1, padding='same', activation = 'relu', strides = strides)(inputs)
    outputs = Concatenate()([x1, x2, x3])
    outputs = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.8)(outputs)    
    return outputs

In [12]:
def MK(inputs, units = [12, 24, 48], strides = 1):
    """
    naive google inception block
    """
    x1 = Conv2D(units[0], 7, padding='same', activation = 'relu', strides = strides)(inputs)
    x2 = Conv2D(units[1], 5, padding='same', activation = 'relu', strides = strides)(inputs)
    x3 = Conv2D(units[2], 3, padding='same', activation = 'relu', strides = strides)(inputs)
    outputs = Concatenate()([x1, x2, x3])
    outputs = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.8)(outputs)    
    return outputs

In [3]:
def _MALDINet(input_shape,  
               n_outputs = 1, 
               n_inception = 2,
               dense_layers = [256, 128, 64, 32], 
               dense_avf = 'relu', 
               dropout = 0,
               last_avf = 'softmax'):

    """
    parameters
    ----------------------
    input_shape: w, h, c
    n_outputs: output units
    n_inception: number of the inception layers
    dense_layers: list, how many dense layers and units
    dense_avf: activation function for dense layers
    last_avf: activation function for last layer
    dropout: dropout of the dense layers
    """
    tf.keras.backend.clear_session()
    assert len(input_shape) == 3
    inputs = Input(input_shape)
    
    incept = MK(inputs=inputs)

    for i in range(n_inception):
        incept = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(incept) #p1
        incept = Inception(incept, strides = 1, units = 32*(2**i))

    #flatten
    x = GlobalMaxPool2D()(incept)
    
    ## dense layer
    for units in dense_layers:
        x = Dense(units, activation = dense_avf)(x)
        if dropout:
            x = Dropout(rate = dropout)(x)

    #last layer
    outputs = Dense(n_outputs,activation = last_avf)(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    
    return model

In [59]:
drug = 'cip'
#Labels loading
Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
train_Y = np.zeros((len(Labels),2))                                    #将标签以独热编码存起来 
test_Y = np.zeros(len(Labels))                                         #用于分层测试的标签

i = 0
for label in Labels[0]:
    if label == 0:
        train_Y[i,0] = 1
        test_Y[i] = 0
    else:
        train_Y[i,1] = 1
        test_Y[i] = 1
    i += 1

y = train_Y.astype(int)
Y = test_Y.astype(int)
X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))

In [61]:
# 1. 准备数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 编译模型
model = _MALDINet(X.shape[1:], n_outputs = y.shape[-1], dropout = 0)
opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
auc_metric = tf.keras.metrics.AUC(curve='ROC')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])

# 3. 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=100)

# 计算测试集上的 AUC
y_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, y_pred)
test_auc

Epoch 1/100
62/62 [==============================] - ETA: 2:05 - loss: 1.5443 - auc: 0.619 - ETA: 1s - loss: 0.9606 - auc: 0.6283  - ETA: 1s - loss: 0.8356 - auc: 0.644 - ETA: 1s - loss: 0.7871 - auc: 0.667 - ETA: 1s - loss: 0.7583 - auc: 0.672 - ETA: 1s - loss: 0.7382 - auc: 0.673 - ETA: 1s - loss: 0.7165 - auc: 0.682 - ETA: 1s - loss: 0.7003 - auc: 0.690 - ETA: 1s - loss: 0.6952 - auc: 0.690 - ETA: 1s - loss: 0.6900 - auc: 0.688 - ETA: 1s - loss: 0.6829 - auc: 0.691 - ETA: 1s - loss: 0.6780 - auc: 0.693 - ETA: 1s - loss: 0.6699 - auc: 0.698 - ETA: 0s - loss: 0.6646 - auc: 0.699 - ETA: 0s - loss: 0.6578 - auc: 0.704 - ETA: 0s - loss: 0.6567 - auc: 0.703 - ETA: 0s - loss: 0.6555 - auc: 0.702 - ETA: 0s - loss: 0.6536 - auc: 0.702 - ETA: 0s - loss: 0.6537 - auc: 0.701 - ETA: 0s - loss: 0.6524 - auc: 0.702 - ETA: 0s - loss: 0.6502 - auc: 0.703 - ETA: 0s - loss: 0.6480 - auc: 0.705 - ETA: 0s - loss: 0.6478 - auc: 0.704 - ETA: 0s - loss: 0.6453 - auc: 0.706 - ETA: 0s - loss: 0.6434 - auc: 0

pip

In [55]:
drug = 'pip'
#Labels loading
Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
train_Y = np.zeros((len(Labels),2))                                    #将标签以独热编码存起来 
test_Y = np.zeros(len(Labels))                                         #用于分层测试的标签

i = 0
for label in Labels[0]:
    if label == 0:
        train_Y[i,0] = 1
        test_Y[i] = 0
    else:
        train_Y[i,1] = 1
        test_Y[i] = 1
    i += 1

y = train_Y.astype(int)
Y = test_Y.astype(int)

X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))

In [58]:
# 1. 准备数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 编译模型
model = _MALDINet(X.shape[1:], n_outputs = y.shape[-1], dropout = 0.3)
opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
auc_metric = tf.keras.metrics.AUC(curve='ROC')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])

# 3. 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=80)

# 计算测试集上的 AUC
y_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, y_pred)
test_auc

Epoch 1/80
60/60 [==============================] - ETA: 1:40 - loss: 1.4295 - auc: 0.839 - ETA: 1s - loss: 1.5234 - auc: 0.8192  - ETA: 1s - loss: 1.5401 - auc: 0.812 - ETA: 1s - loss: 1.3426 - auc: 0.833 - ETA: 1s - loss: 1.2790 - auc: 0.850 - ETA: 1s - loss: 1.1607 - auc: 0.861 - ETA: 1s - loss: 1.0528 - auc: 0.872 - ETA: 1s - loss: 1.0466 - auc: 0.868 - ETA: 1s - loss: 0.9890 - auc: 0.875 - ETA: 1s - loss: 0.9277 - auc: 0.882 - ETA: 1s - loss: 0.9082 - auc: 0.883 - ETA: 1s - loss: 0.8820 - auc: 0.885 - ETA: 1s - loss: 0.8613 - auc: 0.885 - ETA: 1s - loss: 0.8466 - auc: 0.885 - ETA: 0s - loss: 0.8194 - auc: 0.886 - ETA: 0s - loss: 0.8069 - auc: 0.885 - ETA: 0s - loss: 0.7917 - auc: 0.886 - ETA: 0s - loss: 0.7751 - auc: 0.886 - ETA: 0s - loss: 0.7511 - auc: 0.888 - ETA: 0s - loss: 0.7375 - auc: 0.889 - ETA: 0s - loss: 0.7168 - auc: 0.891 - ETA: 0s - loss: 0.7079 - auc: 0.891 - ETA: 0s - loss: 0.6910 - auc: 0.893 - ETA: 0s - loss: 0.6822 - auc: 0.893 - ETA: 0s - loss: 0.6709 - auc: 0.

0.7267173963602535

tob

In [32]:
drug = 'tob'
#Labels loading
Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
train_Y = np.zeros((len(Labels),2))                                    #将标签以独热编码存起来 
test_Y = np.zeros(len(Labels))                                         #用于分层测试的标签

i = 0
for label in Labels[0]:
    if label == 0:
        train_Y[i,0] = 1
        test_Y[i] = 0
    else:
        train_Y[i,1] = 1
        test_Y[i] = 1
    i += 1

y = train_Y.astype(int)
Y = test_Y.astype(int)

X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))

In [34]:
# 1. 准备数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 编译模型
model = _MALDINet(X.shape[1:], n_outputs = y.shape[-1], dropout = 0.3)
opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
auc_metric = tf.keras.metrics.AUC(curve='ROC')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])

# 3. 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=60)

# 计算测试集上的 AUC
y_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, y_pred)
test_auc

Epoch 1/60
61/61 [==============================] - ETA: 2:14 - loss: 3.7623 - auc: 0.550 - ETA: 1s - loss: 2.9581 - auc: 0.6228  - ETA: 1s - loss: 2.4312 - auc: 0.666 - ETA: 1s - loss: 2.2440 - auc: 0.683 - ETA: 1s - loss: 2.0873 - auc: 0.701 - ETA: 1s - loss: 1.9541 - auc: 0.715 - ETA: 1s - loss: 1.8148 - auc: 0.734 - ETA: 1s - loss: 1.7766 - auc: 0.734 - ETA: 1s - loss: 1.7113 - auc: 0.739 - ETA: 1s - loss: 1.6195 - auc: 0.752 - ETA: 1s - loss: 1.5648 - auc: 0.754 - ETA: 1s - loss: 1.5257 - auc: 0.756 - ETA: 1s - loss: 1.4561 - auc: 0.764 - ETA: 1s - loss: 1.4043 - auc: 0.768 - ETA: 0s - loss: 1.3611 - auc: 0.772 - ETA: 0s - loss: 1.3225 - auc: 0.775 - ETA: 0s - loss: 1.2787 - auc: 0.779 - ETA: 0s - loss: 1.2391 - auc: 0.783 - ETA: 0s - loss: 1.1996 - auc: 0.787 - ETA: 0s - loss: 1.1698 - auc: 0.789 - ETA: 0s - loss: 1.1562 - auc: 0.788 - ETA: 0s - loss: 1.1396 - auc: 0.788 - ETA: 0s - loss: 1.1189 - auc: 0.790 - ETA: 0s - loss: 1.0896 - auc: 0.793 - ETA: 0s - loss: 1.0685 - auc: 0.

0.7048648648648649

In [35]:
drug = 'ceft'
#Labels loading
Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
train_Y = np.zeros((len(Labels),2))                                    #将标签以独热编码存起来 
test_Y = np.zeros(len(Labels))                                         #用于分层测试的标签

i = 0
for label in Labels[0]:
    if label == 0:
        train_Y[i,0] = 1
        test_Y[i] = 0
    else:
        train_Y[i,1] = 1
        test_Y[i] = 1
    i += 1

y = train_Y.astype(int)
Y = test_Y.astype(int)

X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))

In [36]:
# 1. 准备数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 编译模型
model = _MALDINet(X.shape[1:], n_outputs = y.shape[-1], dropout = 0.3)
opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
auc_metric = tf.keras.metrics.AUC(curve='ROC')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])

# 3. 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=60)

# 计算测试集上的 AUC
y_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, y_pred)
test_auc

Epoch 1/60
62/62 [==============================] - ETA: 1:35 - loss: 2.5993 - auc: 0.751 - ETA: 1s - loss: 2.3468 - auc: 0.7068  - ETA: 1s - loss: 2.2396 - auc: 0.682 - ETA: 1s - loss: 2.1060 - auc: 0.672 - ETA: 1s - loss: 2.1326 - auc: 0.642 - ETA: 1s - loss: 2.0486 - auc: 0.640 - ETA: 1s - loss: 1.9597 - auc: 0.633 - ETA: 1s - loss: 1.8721 - auc: 0.633 - ETA: 1s - loss: 1.7743 - auc: 0.639 - ETA: 1s - loss: 1.6992 - auc: 0.644 - ETA: 1s - loss: 1.6202 - auc: 0.652 - ETA: 1s - loss: 1.5596 - auc: 0.659 - ETA: 1s - loss: 1.5217 - auc: 0.660 - ETA: 0s - loss: 1.4917 - auc: 0.663 - ETA: 0s - loss: 1.4612 - auc: 0.664 - ETA: 0s - loss: 1.4290 - auc: 0.665 - ETA: 0s - loss: 1.3779 - auc: 0.670 - ETA: 0s - loss: 1.3593 - auc: 0.667 - ETA: 0s - loss: 1.3194 - auc: 0.670 - ETA: 0s - loss: 1.3021 - auc: 0.669 - ETA: 0s - loss: 1.2831 - auc: 0.668 - ETA: 0s - loss: 1.2563 - auc: 0.670 - ETA: 0s - loss: 1.2366 - auc: 0.670 - ETA: 0s - loss: 1.2146 - auc: 0.672 - ETA: 0s - loss: 1.1988 - auc: 0.

0.8702636983535439

In [37]:
drug = 'cefe'
#Labels loading
Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
train_Y = np.zeros((len(Labels),2))                                    #将标签以独热编码存起来 
test_Y = np.zeros(len(Labels))                                         #用于分层测试的标签

i = 0
for label in Labels[0]:
    if label == 0:
        train_Y[i,0] = 1
        test_Y[i] = 0
    else:
        train_Y[i,1] = 1
        test_Y[i] = 1
    i += 1

y = train_Y.astype(int)
Y = test_Y.astype(int)

X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))

In [38]:
# 1. 准备数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 编译模型
model = _MALDINet(X.shape[1:], n_outputs = y.shape[-1], dropout = 0.3)
opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
auc_metric = tf.keras.metrics.AUC(curve='ROC')
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])

# 3. 训练模型
history = model.fit(X_train, y_train, batch_size=64, epochs=60)

# 计算测试集上的 AUC
y_pred = model.predict(X_test)
test_auc = roc_auc_score(y_test, y_pred)
test_auc

Epoch 1/60
62/62 [==============================] - ETA: 1:40 - loss: 12.9310 - auc: 0.14 - ETA: 1s - loss: 10.0996 - auc: 0.2284 - ETA: 1s - loss: 8.2433 - auc: 0.2440 - ETA: 1s - loss: 6.8946 - auc: 0.270 - ETA: 1s - loss: 6.1261 - auc: 0.298 - ETA: 1s - loss: 5.4367 - auc: 0.325 - ETA: 1s - loss: 4.8548 - auc: 0.359 - ETA: 1s - loss: 4.4508 - auc: 0.384 - ETA: 1s - loss: 4.0789 - auc: 0.420 - ETA: 1s - loss: 3.7761 - auc: 0.450 - ETA: 1s - loss: 3.5783 - auc: 0.467 - ETA: 1s - loss: 3.3888 - auc: 0.483 - ETA: 1s - loss: 3.1901 - auc: 0.506 - ETA: 0s - loss: 3.0427 - auc: 0.520 - ETA: 0s - loss: 2.8960 - auc: 0.532 - ETA: 0s - loss: 2.7716 - auc: 0.542 - ETA: 0s - loss: 2.6525 - auc: 0.553 - ETA: 0s - loss: 2.5630 - auc: 0.558 - ETA: 0s - loss: 2.4898 - auc: 0.565 - ETA: 0s - loss: 2.4318 - auc: 0.569 - ETA: 0s - loss: 2.3623 - auc: 0.573 - ETA: 0s - loss: 2.2925 - auc: 0.579 - ETA: 0s - loss: 2.2239 - auc: 0.586 - ETA: 0s - loss: 2.1787 - auc: 0.589 - ETA: 0s - loss: 2.1279 - auc: 0

0.8655577919415558

In [ ]:
import itertools

In [ ]:
DRUGS = ['cip', 'ceft', 'cefe', 'pip', 'tob']
DROPOUT = [0, 0.1, 0.2, 0.3, 0.4]
EPOCHS = [40, 60, 80, 100]

In [ ]:
for drug in DRUGS :
    #加载标签及图像数据
    Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
    train_Y = np.zeros((len(Labels),2))
    i = 0
    for label in Labels[0]:
        if label == 0:
            train_Y[i,0] = 1
        else:
            train_Y[i,1] = 1
        i += 1
    y = train_Y.astype(int)
    X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))
    #分割数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for dropout, epochs in itertools.product(DROPOUT, EPOCHS):
        #编译及训练模型
        model = _MALDINet(X.shape[1:], n_outputs=y.shape[-1], dropout=dropout)
        opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        auc_metric = tf.keras.metrics.AUC(curve='ROC')
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])
        history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, verbose=1)
        # 计算测试集AUC并记录
        y_pred = model.predict(X_test)
        test_auc = roc_auc_score(y_test, y_pred)
        with open("output_kpn.txt", "a") as file:
            file.write(drug + ' dropout = ' + str(dropout) + ' epochs = ' + str(epochs) + ' auc = ' + str(test_auc) + "\n")